# Exploratory Data 
Using captum following [this tutorial](https://captum.ai/tutorials/House_Prices_Regression_Interpret) for regression

In [ ]:
import json
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from captum.attr import LayerConductance, LayerActivation, LayerIntegratedGradients
from captum.attr import IntegratedGradients, DeepLift, GradientShap, NoiseTunnel, FeatureAblation

In [ ]:
model_info = json.load(open("configs/model_info.json"))
pv_info = json.load(open("configs/pv_info.json"))
nn_transform_info = json.load(open("configs/normalization.json"))

In [ ]:
from transformers import create_sim_to_nn_transformers
from transformed_model import KeyedTransformedModel
# get transformers for normailzation into NN
nn_input_transformer, nn_output_transformer = create_sim_to_nn_transformers(
    "configs/normalization.json"
)
test_min = torch.tensor(model_info["train_input_mins"])
test_max = torch.tensor(model_info["train_input_maxs"]).unsqueeze(0)

model = torch.load("torch_model.pt").double()

# define the NN surrogate that contains the NN, the input/output transformers for
# simulation units
surrogate = KeyedTransformedModel(
    model,
    nn_input_transformer,
    nn_output_transformer,
    model_info["model_in_list"],
    model_info["model_out_list"]
)


In [ ]:
raw_x_data = np.load("data/x_raw_small.npy", allow_pickle=True)
raw_y_data = np.load("data/y_raw_small.npy", allow_pickle=True).astype('float')

x_df = pd.DataFrame(raw_x_data, columns=model_info['model_in_list'])
y_df = pd.DataFrame(raw_y_data, columns=model_info['model_out_list'])

preds = surrogate(torch.tensor(raw_x_data).double())

Let's pick one of the output variables to see how it varies with each input. Once we have the process nailed down we can repeat this for the other four outputs.

In [ ]:
output_var = 'sigma_x'
feature_names = model_info['model_in_list']
print(model_info['model_out_list'])

In [ ]:
fig, axs = plt.subplots(nrows = 4, ncols=4, sharey=True, figsize=(20,15))
for i, (ax, col) in enumerate(zip(axs.flat, feature_names)):    
    x = raw_x_data[:,i]
    pf = np.polyfit(x, raw_y_data[:,0], 1)
    p = np.poly1d(pf)

    ax.plot(x, raw_y_data[:,0], 'o')
    ax.plot(x, p(x),"r--")

    ax.set_title(col + ' vs sigma_x')
    ax.set_xlabel(col)
# axs[:,0].set_ylabel('sigma_x')

fig.tight_layout()
plt.show()

In [ ]:
for idx, output_name in enumerate(model_info['model_out_list']):
    ig = IntegratedGradients(model)
    ig_nt = NoiseTunnel(ig)
    dl = DeepLift(model)
    gs = GradientShap(model)
    fa = FeatureAblation(model)

    X_test = torch.Tensor(raw_x_data).double()

    # NOTE for multi-output problems you need to pass the target value of the output you're interested
    # in, for example here we are interested in the first prediction (sigma_x) so we pass target=0

    ig_attr_test = ig.attribute(X_test, target=idx, n_steps=50)
    ig_nt_attr_test = ig_nt.attribute(X_test, target=idx)
    dl_attr_test = dl.attribute(X_test, target=idx)
    fa_attr_test = fa.attribute(X_test, target=idx)

    # plot feature importances
    x_axis_data = np.arange(X_test.shape[1])
    x_axis_data_labels = list(map(lambda idx: feature_names[idx], x_axis_data))

    ig_attr_test_sum = ig_attr_test.detach().numpy().sum(0)
    ig_attr_test_norm_sum = ig_attr_test_sum / np.linalg.norm(ig_attr_test_sum, ord=1)

    ig_nt_attr_test_sum = ig_nt_attr_test.detach().numpy().sum(0)
    ig_nt_attr_test_norm_sum = ig_nt_attr_test_sum / np.linalg.norm(ig_nt_attr_test_sum, ord=1)

    dl_attr_test_sum = dl_attr_test.detach().numpy().sum(0)
    dl_attr_test_norm_sum = dl_attr_test_sum / np.linalg.norm(dl_attr_test_sum, ord=1)

    fa_attr_test_sum = fa_attr_test.detach().numpy().sum(0)
    fa_attr_test_norm_sum = fa_attr_test_sum / np.linalg.norm(fa_attr_test_sum, ord=1)

    width = 0.14
    legends = ['Int Grads', 'Int Grads w/SmoothGrad','DeepLift', 'Feature Ablation']

    plt.figure(figsize=(20, 10))

    ax = plt.subplot()
    ax.set_title('Comparing input feature importances across multiple algorithms and learned weights')
    ax.set_ylabel('Attributions')

    ax.bar(x_axis_data, ig_attr_test_norm_sum, width, align='center', alpha=0.8, color='#eb5e7c')
    ax.bar(x_axis_data + width, ig_nt_attr_test_norm_sum, width, align='center', alpha=0.7, color='#A90000')
    ax.bar(x_axis_data + 2 * width, dl_attr_test_norm_sum, width, align='center', alpha=0.6, color='#34b8e0')
    ax.bar(x_axis_data + 3 * width, fa_attr_test_norm_sum, width, align='center', alpha=1.0, color='#49ba81')
    ax.autoscale_view()
    plt.tight_layout()

    ax.set_xticks(x_axis_data + 0.5)
    ax.set_xticklabels(x_axis_data_labels)
    plt.xticks(rotation=90)
    plt.legend(legends, loc=3)
    plt.title(output_name)
    plt.show()

In [ ]:
ig = IntegratedGradients(model)
ig_nt = NoiseTunnel(ig)
dl = DeepLift(model)
gs = GradientShap(model)
fa = FeatureAblation(model)

X_test = torch.Tensor(raw_x_data).double()

# NOTE for multi-output problems you need to pass the target value of the output you're interested
# in, for example here we are interested in the first prediction (sigma_x) so we pass target=0

ig_attr_test = ig.attribute(X_test, target=0, n_steps=50)
ig_nt_attr_test = ig_nt.attribute(X_test, target=0)
dl_attr_test = dl.attribute(X_test, target=0)
# gs_attr_test = gs.attribute(X_test, target=0, X_train)
fa_attr_test = fa.attribute(X_test, target=0)

In [ ]:
x_axis_data = np.arange(X_test.shape[1])
x_axis_data_labels = list(map(lambda idx: feature_names[idx], x_axis_data))

ig_attr_test_sum = ig_attr_test.detach().numpy().sum(0)
ig_attr_test_norm_sum = ig_attr_test_sum / np.linalg.norm(ig_attr_test_sum, ord=1)

ig_nt_attr_test_sum = ig_nt_attr_test.detach().numpy().sum(0)
ig_nt_attr_test_norm_sum = ig_nt_attr_test_sum / np.linalg.norm(ig_nt_attr_test_sum, ord=1)

dl_attr_test_sum = dl_attr_test.detach().numpy().sum(0)
dl_attr_test_norm_sum = dl_attr_test_sum / np.linalg.norm(dl_attr_test_sum, ord=1)

fa_attr_test_sum = fa_attr_test.detach().numpy().sum(0)
fa_attr_test_norm_sum = fa_attr_test_sum / np.linalg.norm(fa_attr_test_sum, ord=1)

width = 0.14
legends = ['Int Grads', 'Int Grads w/SmoothGrad','DeepLift', 'GradientSHAP', 'Feature Ablation', 'Weights']

plt.figure(figsize=(20, 10))

ax = plt.subplot()
ax.set_title('Comparing input feature importances across multiple algorithms and learned weights')
ax.set_ylabel('Attributions')

ax.bar(x_axis_data, ig_attr_test_norm_sum, width, align='center', alpha=0.8, color='#eb5e7c')
ax.bar(x_axis_data + width, ig_nt_attr_test_norm_sum, width, align='center', alpha=0.7, color='#A90000')
ax.bar(x_axis_data + 2 * width, dl_attr_test_norm_sum, width, align='center', alpha=0.6, color='#34b8e0')
ax.bar(x_axis_data + 3 * width, fa_attr_test_norm_sum, width, align='center', alpha=1.0, color='#49ba81')
ax.autoscale_view()
plt.tight_layout()

ax.set_xticks(x_axis_data + 0.5)
ax.set_xticklabels(x_axis_data_labels)
plt.xticks(rotation=90)
plt.legend(legends, loc=3)
plt.show()